In [69]:
import pandas as pd

files = ['abu sayyaf_2012-01-01_2014-09-01_Basile.csv',  # finished
         'afghanistan_2012-01-01_2014-09-01_Basile.csv', # in progress
         'agro_2012-01-01_2014-09-01_Basile.csv',        # in progress
         'al-shabaab_2012-01-01_2014-09-01_Basile.csv',  # finished
         'ammonium nitrate_2012-01-01_2014-09-01_Basile.csv', # finished
         'nationalism_2012-01-01_2014-09-01_Basile.csv',
         
         'eco-terrorism_2012-01-01_2014-09-01_Basile.csv',    # finished
         'tamil tigers_2012-01-01_2014-09-01_Basile.csv',     # finished
         'weapons-grade_2012-01-01_2014-09-01_Basile.csv',    # finished
        ]

file = files[5]
print(file)
df = pd.read_csv(file, usecols=keep_columns, dtype=defined_types)
print(f'Oldest tweet dates : {df.iloc[df.index[-1]].date}')

nationalism_2012-01-01_2014-09-01_Basile.csv
Oldest tweet dates : 2012-01-01


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6168416 entries, 0 to 6168415
Data columns (total 36 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int64  
 1   conversation_id  int64  
 2   created_at       object 
 3   date             object 
 4   time             object 
 5   timezone         int64  
 6   user_id          int64  
 7   username         object 
 8   name             object 
 9   place            object 
 10  tweet            object 
 11  language         object 
 12  mentions         object 
 13  urls             object 
 14  photos           object 
 15  replies_count    int64  
 16  retweets_count   int64  
 17  likes_count      int64  
 18  hashtags         object 
 19  cashtags         object 
 20  link             object 
 21  retweet          bool   
 22  quote_url        object 
 23  video            int64  
 24  thumbnail        object 
 25  near             float64
 26  geo              float64
 27  source      

In [2]:
searches = ["abu sayyaf" ,
 "afghanistan" ,
 "agro" ,
 "al-qaeda" ,
 "al-qaeda in the arabian peninsula" ,
 "al-qaeda in the islamic maghreb" ,
 "al-shabaab" ,
 "ammonium nitrate" ,
 "attack" ,
 "biological weapon" ,
 "car bomb" ,
 "chemical weapon" ,
 "conventional weapon" ,
 "dirty bomb" ,
 "eco-terrorism" ,
 "environmental terrorism" ,
 "euskadi ta askatasuna" ,
 "extremism" ,
 "farc" ,
 "fundamentalism" ,
 "hamas" ,
 "hezbollah" ,
 "improvised explosive device" ,
 "iran" ,
 "iraq" ,
 "irish republican army" ,
 "islamist" ,
 "jihad" ,
 "nationalism" ,
 "nigeria" ,
 "nuclear" ,
 "nuclear enrichment" ,
 "pakistan" ,
 "palestine liberation front" ,
 "pirates" ,
 "plo" ,
 "political radicalism" ,
 "recruitment" ,
 "somalia" ,
 "suicide attack" ,
 "suicide bomber" ,
 "taliban" ,
 "tamil tigers" ,
 "tehrik-i-taliban pakistan" ,
 "terror" ,
 "terrorism" ,
 "weapons-grade" ,
 "yemen"]

In [24]:
defined_types = {'id': 'int64',
 'conversation_id': 'int64',
 'date': 'str',
 'time': 'str',
 'user_id': 'int64',
 'username': 'str',
 'name': 'str',
 'tweet': 'str',
 'language': 'category',
 'mentions': 'str', # maybe get rid of
 'urls': 'str',
 'photos': 'str',
 'replies_count': 'int64',
 'retweets_count': 'int64',
 'likes_count': 'int64',
 'hashtags': 'str',
 'link': 'str',
 'retweet': 'bool', # is it a retweet ?
 'video': 'bool',
 'thumbnail': 'str',
 'reply_to': 'str'}

date_cols = ['created_at']

remove_cols = ['near',
               'geo',
               'source',
               'user_rt_id',
               'user_rt',
               'retweet_id',
               'retweet_date',
               'translate',
               'trans_src',
               'trans_dest']
keep_columns = ['id',
 'conversation_id',
 'created_at',
 'date',
 'time',
 'user_id',
 'username',
 'name',
 'tweet',
 'language',
 'mentions',
 'urls',
 'photos',
 'replies_count',
 'retweets_count',
 'likes_count',
 'hashtags',
 'link',
 'retweet',
 'video',
 'thumbnail',
 'reply_to']

In [ ]:
# get status

import os
from tqdm import tqdm

dir_files = os.listdir('.')
data_files = [f for f in dir_files if f.endswith('.csv')]
total_count = len(data_files)
count_finished = 0
meta_datasets = {}
keywords = []
mems = []
lengths = []
states = []
last_dates = []
for f in tqdm(data_files):
#     print(f)
    df = pd.read_csv(f, usecols=keep_columns)
    keyword = f.split('_')[0]
    
    mem = df.memory_usage().sum()
    length = len(df.index)
    state = 'finished' if df.iloc[df.index[-1]].date[:-3] == '2012-01' else 'in progress'
    count_finished += 1 if state == 'finished' else 0
    last_date = df.iloc[df.index[-1]].date
    keywords.append(keyword)
    mems.append(mem)
    lengths.append(length)
    states.append(state)
    last_dates.append(last_date)
    
    meta_datasets[keyword] = {'mem': mem, 'length': length, 'state': state, 'last_date': last_date}

    
stats = {'keyword': keywords, 'mem': mems, 'length': lengths, 'state': states, 'last_date': last_dates}
print(f'Finished : {count_finished}/{total_count}')

with open('stats.txt', 'w') as stats_file:
    stats_file.write(repr(meta_datasets))


 16%|█▋        | 9/55 [03:05<29:40, 38.70s/it]

In [111]:
# sum up all things :
stats_df = pd.DataFrame.from_dict(stats).sort_values('keyword')
all_articles_df = pd.DataFrame(searches, columns=['keyword'])
final_files = [f.split('_')[0] for f in os.listdir('./data') if f.endswith('.gz')]
final_df = pd.DataFrame.from_dict({'keyword': final_files, 'final': ['ok']*len(final_files)})

# join the dataframes
combined_df = stats_df.merge(all_articles_df, how='outer', left_on='keyword', right_on='keyword').sort_values('keyword').reset_index(drop=True)
# combined_df.to_csv('progress.csv')
combined_df = combined_df.merge(final_df, how='outer', left_on='keyword', right_on='keyword').sort_values('keyword').reset_index(drop=True)
combined_df

,keyword,mem,length,state,last_date,final
0,abu sayyaf,6.696922e+06,39626.0,finished,2012-01-01,ok
1,afghanistan,2.805413e+07,166000.0,finished,2012-01-01,ok
2,afghanistan,1.042462e+09,6168416.0,in progress,2012-02-01,ok
3,agro,1.888908e+08,1117696.0,finished,2012-01-01,ok
4,al-qaeda,1.330684e+08,787386.0,finished,2012-01-01,ok
5,al-qaeda in the arabian peninsula,2.634300e+05,1558.0,finished,2012-01-02,ok
6,al-qaeda in the islamic maghreb,1.892390e+05,1119.0,finished,2012-01-04,ok
7,al-shabaab,2.000516e+07,118373.0,finished,2012-01-01,ok
8,ammonium nitrate,2.427137e+06,14361.0,finished,2012-01-01,ok
9,attack,1.130101e+09,6686989.0,in progress,2014-04-05,NaN


In [101]:
disk_size = [os.stat(file).st_size for file in data_files]
print(f'Memory (RAM) occupation : {stats_df.mem.sum()/1000000:.1f}MB')
print(f'Disk occupation : {sum(disk_size)/1000000:.1f}MB')

Memory (RAM) occupation : 11013.8MB
Disk occupation : 25653.4MB


In [47]:
term = "\"abu sayyaf\" OR \"afghanistan\" OR \"agro\" OR \"al-qaeda\" OR \"al-qaeda in the arabian peninsula\" OR \"al-qaeda in the islamic maghreb\" OR \"al-shabaab\" OR \"ammonium nitrate\" OR \"attack\" OR \"biological weapon\" OR \"car bomb\" OR \"chemical weapon\" OR \"conventional weapon\" OR \"dirty bomb\" OR \"eco-terrorism\" OR \"environmental terrorism\" OR \"euskadi ta askatasuna\" OR \"extremism\" OR \"farc\" OR \"fundamentalism\" OR \"hamas\" OR \"hezbollah\" OR \"improvised explosive device\" OR \"iran\" OR \"iraq\" OR \"irish republican army\" OR \"islamist\" OR \"jihad\" OR \"nationalism\" OR \"nigeria\" OR \"nuclear\" OR \"nuclear enrichment\" OR \"pakistan\" OR \"palestine liberation front\" OR \"pirates\" OR \"plo\" OR \"political radicalism\" OR \"recruitment\" OR \"somalia\" OR \"suicide attack\" OR \"suicide bomber\" OR \"taliban\" OR \"tamil tigers\" OR \"tehrik-i-taliban pakistan\" OR \"terror\" OR \"terrorism\" OR \"weapons-grade\" OR \"yemen\""
print('- ' + '\n- '.join(searches))

- abu sayyaf
- afghanistan
- agro
- al-qaeda
- al-qaeda in the arabian peninsula
- al-qaeda in the islamic maghreb
- al-shabaab
- ammonium nitrate
- attack
- biological weapon
- car bomb
- chemical weapon
- conventional weapon
- dirty bomb
- eco-terrorism
- environmental terrorism
- euskadi ta askatasuna
- extremism
- farc
- fundamentalism
- hamas
- hezbollah
- improvised explosive device
- iran
- iraq
- irish republican army
- islamist
- jihad
- nationalism
- nigeria
- nuclear
- nuclear enrichment
- pakistan
- palestine liberation front
- pirates
- plo
- political radicalism
- recruitment
- somalia
- suicide attack
- suicide bomber
- taliban
- tamil tigers
- tehrik-i-taliban pakistan
- terror
- terrorism
- weapons-grade
- yemen


In [93]:
%%time
import pandas as pd
# joining dataframes
# , parse_dates=date_cols
df_1 = pd.read_csv('somalia_2012-01-01_2014-09-01_Basile.csv', usecols=keep_columns, dtype=defined_types)
df_2 = pd.read_csv('somalia_2012-01-01_2013-12-27_Basile.csv', usecols=keep_columns, dtype=defined_types)

print(f'Number of entries in df_1 is : {len(df_1)}')
print(f'Number of entries in df_2 is : {len(df_2)}')
print(f'Last date in df_1 is : {df_1.iloc[df_1.index[-1]].date}')
df_1 = df_1.append(df_2)
df_1 = df_1.reset_index(drop=True)
print(f'Number of entries in df_1 after append is : {len(df_1)}')
print(f'Last date in df_1 after append is : {df_1.iloc[df_1.index[-1]].date}')
print(f'Last date in df_2 is : {df_2.iloc[df_2.index[-1]].date}')

# remove duplicate
print(f'Number of entries before duplicates : {len(df_save)}')
df_1 = df_1.drop_duplicates()
print(f'Number of entries after duplicates : {len(df_save)}')

# save the appended dataframe
df_1.to_csv('data/somalia_full.gz', index=False, encoding='utf-8')

Number of entries in df_1 is : 501427
Number of entries in df_2 is : 1575392
Last date in df_1 is : 2013-12-27
Number of entries in df_1 after append is : 2076819
Last date in df_1 after append is : 2012-01-01
Last date in df_2 is : 2012-01-01
Number of entries before duplicates : 3858642
Number of entries after duplicates : 3858642
CPU times: user 1min 31s, sys: 12.7 s, total: 1min 43s
Wall time: 1min 53s


In [115]:
%%time
# open dataframe
df_save = pd.read_csv('islamist_2012-01-01_2014-09-01_Basile.csv', usecols=keep_columns, dtype=defined_types)#, parse_dates=['created_at'])

# remove duplicate
print(f'Number of entries before duplicates : {len(df_save)}')
df_save = df_save.drop_duplicates()
print(f'Number of entries after duplicates : {len(df_save)}')

# save the appended dataframe
df_save.to_csv('data/islamist_full.gz', index=False, encoding='utf-8')

Number of entries before duplicates : 1004562
Number of entries after duplicates : 1004562
CPU times: user 46.2 s, sys: 2.04 s, total: 48.3 s
Wall time: 50.6 s


In [96]:
df_1.info()
# df_save.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2076819 entries, 0 to 2076818
Data columns (total 22 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               int64 
 1   conversation_id  int64 
 2   created_at       object
 3   date             object
 4   time             object
 5   user_id          int64 
 6   username         object
 7   name             object
 8   tweet            object
 9   language         object
 10  mentions         object
 11  urls             object
 12  photos           object
 13  replies_count    int64 
 14  retweets_count   int64 
 15  likes_count      int64 
 16  hashtags         object
 17  link             object
 18  retweet          bool  
 19  video            bool  
 20  thumbnail        object
 21  reply_to         object
dtypes: bool(2), int64(6), object(14)
memory usage: 336.7+ MB


In [23]:
df_save.created_at.sample(10)
# df_1.info(null_counts=True)
# dict(df_1.dtypes)
# df_1[df_1.video==1].sample(10)
# len(a)

27565   2013-03-23 07:38:57+01:00
19447   2014-03-03 15:44:56+01:00
34675   2012-07-30 05:46:27+02:00
3137    2014-07-23 14:21:09+02:00
6168    2014-06-04 20:26:16+02:00
35476   2012-07-27 06:31:00+02:00
2181    2014-07-28 14:17:57+02:00
27959   2013-03-23 03:58:22+01:00
32888   2012-10-21 03:12:44+02:00
31687   2013-02-03 04:39:32+01:00
Name: created_at, dtype: datetime64[ns, tzlocal()]

In [21]:
# (df_1.id != df_1.conversation_id).sum()
df_1.columns.to_list()

['id',
 'conversation_id',
 'created_at',
 'date',
 'time',
 'timezone',
 'user_id',
 'username',
 'name',
 'place',
 'tweet',
 'language',
 'mentions',
 'urls',
 'photos',
 'replies_count',
 'retweets_count',
 'likes_count',
 'hashtags',
 'cashtags',
 'link',
 'retweet',
 'quote_url',
 'video',
 'thumbnail',
 'near',
 'geo',
 'source',
 'user_rt_id',
 'user_rt',
 'retweet_id',
 'reply_to',
 'retweet_date',
 'translate',
 'trans_src',
 'trans_dest']